In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import functools
from cryptic.evaluation.evaluations import GeorgeHoQA
from cryptic.models.oai import OpenAIQA
from cryptic.prompts.inject_prompt import inject_prompt

In [ ]:
CSV_FILENAME = "data/examples/qc_1711.csv"
DUMMY_CLUE = "A farmer's son"

Check that injection of clue and num letters into prompt template is working correctly:

In [ ]:
prompt_fn = functools.partial(inject_prompt, "cryptic.prompts.cot.ben_v1")
prompt_fn(DUMMY_CLUE, 5)

In [ ]:
evaluation = GeorgeHoQA(CSV_FILENAME, num_answers=2)

In [ ]:
model = OpenAIQA(prompt_fn=prompt_fn, model_name="text-davinci-003", max_tokens=8)

In [ ]:
df = evaluation.qa_frame.df
df.head(3)

First feed a single clue through the API to look at model output and check that we are correctly parsing responses.

This might need to be tweaked: e.g. removal of newline / space chars etc,
to convert into a standard answer format

In [ ]:
clue, num_letters = evaluation.qa_frame.sample(1).df.iloc[0][["clue", "num_letters"]]

In [ ]:
clue, num_letters

In [ ]:
response = model.get_response(clue, num_letters, num_answers=1)
response

In [ ]:
model.answers_from_response(response)

Once we're confident that answers are being extracted correctly we can run a whole set of clues through

In [ ]:
metrics, answer_df = evaluation.run(model)

In [ ]:
prediction_cols = [c for c in answer_df.columns if c.startswith("prediction")]
answer_df[["clue", "answer"] + prediction_cols + ["correct"]]